In [7]:
import numpy as np

## environment class

In [8]:
class Grid:
    def __init__(self, rows, cols, start):
        self.rows = rows
        self.cols = cols
        self.i = start[0]
        self.j = start[1]

        
    def set(self, rewards, actions):
        # rewards should be a dict of: (i, j): r (row, col): reward
        # actions should be a dict of: (i, j): A (row, col): list of possible actions
        self.rewards = rewards
        self.actions = actions

        
    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

        
    def current_state(self):
        return (self.i, self.j)

    
    def is_terminal(self, s):
        return s not in self.actions

    
    def move(self, action):
        # check if legal move first
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        # return a reward (if any)
        return self.rewards.get((self.i, self.j), 0)

    
    def undo_move(self, action):
        # these are the opposite of what U/D/L/R should normally do
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        # raise an exception if we arrive somewhere we shouldn't be
        # should never happen
        assert(self.current_state() in self.all_states())

    
    def game_over(self):
        # returns true if game is over, else false
        # true if we are in a state where no actions are possible
        return (self.i, self.j) not in self.actions

    
    def all_states(self):
        # possibly buggy but simple way to get all states
        # either a position that has possible next actions
        # or a position that yields a reward
        return set(self.actions.keys()) | set(self.rewards.keys())


def standard_grid():
    # define a grid that describes the reward for arriving at each state
    # and possible actions at each state
    # the grid looks like this
    # x means you can't go there
    # s means start position
    # number means reward at that state
    # .  .  .  1
    # .  x  . -1
    # s  .  .  .
    g = Grid(3, 4, (2, 0))
    rewards = {(0, 3): 1, (1, 3): -1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
    }
    g.set(rewards, actions)
    return g


def negative_grid(step_cost=-0.1):
    # in this game we want to try to minimize the number of moves
    # so we will penalize every move
    g = standard_grid()
    g.rewards.update({
        (0, 0): step_cost,
        (0, 1): step_cost,
        (0, 2): step_cost,
        (1, 0): step_cost,
        (1, 2): step_cost,
        (2, 0): step_cost,
        (2, 1): step_cost,
        (2, 2): step_cost,
        (2, 3): step_cost,
    })
    return g

In [9]:
def print_value(V, g):
    for i in range(g.rows):
        print('----------------------------')
        for j in range(g.cols):
            v = V.get((i,j), 0)
            if v>=0:
                print('| %.2f '%v, end='')
            else:
                print('|%.2f '%v, end='') # negative sign takes up an extra space
        print('|')
    print('----------------------------')
    

def print_policy(P, g):
    for i in range(g.rows):
        print('-----------------')
        for j in range(g.cols):
            a = P.get((i,j), ' ')
            print('| %s '%a, end='')
        print('|')
    print('-----------------')

## Temporal Difference or TD0 method
advantage of this method is that it combines both dynamic and monte carlo programmin
i.e. it overcomes the limitation of mc where we have to wait for an episode to finish to calculate Q value

## optimizing policy using Q learning
The goal is to maximize the Q-value. Q-learning belongs to the off-policy category i.e. the next action is chosen to maximize the next state’s Q-value instead of following the epsilon greedy policy

In [10]:
def random_action(a, eps=0.1):
    # choose given a with probability 1 - eps + eps/4
    # choose some other a' != a with probability eps/4
    p = np.random.random()
    
    # if p < (1 - eps + eps/len(ALL_POSSIBLE_ACTIONS)):
    #   return a
    # else:
    #   tmp = list(ALL_POSSIBLE_ACTIONS)
    #   tmp.remove(a)
    #   return np.random.choice(tmp)
    #
    # this is equivalent to the above
    if p<(1-eps):
        return a
    else:
        return np.random.choice(all_possible_actions)
    
    
def max_dict(d):
    # returns the argmax (key) and max (value) from a dictionary
    max_key = None
    max_value = float('-inf')
    for key, value in d.items():
        if value>max_value:
            max_key = key
            max_value = value
    return max_key, max_value

In [23]:
# let's see how V(s) changes as we get further away from the reward
gamma = 0.9 # discount factor

# set different actions possible for a particular state
all_possible_actions = ['U', 'D', 'R', 'L']

small_enough = 1e-3

alpha = 0.1

# NOTE: if we use the standard grid, there's a good chance we will end up with
# suboptimal policies
# e.g.
# ---------------------------
#   R  |   R  |   R  |      |
# ---------------------------
#   R* |      |   U  |      |
# ---------------------------
#   U  |   R  |   U  |   L  |
# since going R at (1,0) (shown with a *) incurs no cost, it's OK to keep doing that.
# we'll either end up staying in the same spot, or back to the start (2,0), at which
# point we whould then just go back up, or at (0,0), at which point we can continue
# on right.
# instead, let's penalize each movement so the agent will find a shorter route.
#
# grid = standard_grid()
grid = negative_grid(step_cost=-0.1)

# print rewards
print("rewards:")
print_value(grid.rewards, grid)
        
# initialize Q(s, a), alpha(s, a)
# create a dict Q with different states then embedded dictionary with different actions
states = grid.all_states()
Q = {}
update_counts = {}
sa_count = {}
for s in states:
    sa_count[s] = {}
    Q[s] = {}
    for a in all_possible_actions:
        Q[s][a] = 0 # needs to be initialized to something so we can argmax it  
        sa_count[s][a] = 1 # set initial count to be 0
            
# repeat until convergence
# play game for n episodes
for t in range(2000):
    # set first state for starting position
    s = (2, 0)
    grid.set_state(s)
    # choose an action based on max Q value
    a = max_dict(Q[s])[0]
    # loop until one episode is over
    while not grid.game_over():
        a = random_action(a)   # epsilon greedy
        r = grid.move(a)
        s1 = grid.current_state()
        # we need the next action as well since Q(s,a) depends on Q(s',a')
        # if s2 not in policy then it's a terminal state, all Q are 0
        # the difference between SARSA and Q-Learning is with Q-Learning
        # we will use this max[a']{ Q(s',a')} in our update
        # even if we do not end up taking this action in the next step
        a1, max_Q1 = max_dict(Q[s1])
        # Q[s, a] = Q[s, a] + alpha*(r + gamma*max[a']{Q[s', a']} - Q[s, a])
        # here we use alpha as adaptive learning rate like AdaGrad and RMSprop in DNN
        # in this way when epsilon decreases for each episode it may miss the states which have never occur
        # adaptive alpha will be high for such states and hence keeping the balance
        sa_count[s][a] += 0.005
        Q[s][a] = Q[s][a] + (alpha/sa_count[s][a])*(r + gamma*max_Q1 - Q[s][a])
        
        # we would like to know how often Q(s) has been updated too
        update_counts[s] = update_counts.get(s,0) + 1
        
        # set current state as next state
        s = s1
        a = a1
        
# determine the policy from Q*
# initialize policy, V
policy, V = {}, {}
for s in grid.actions.keys():
    policy[s], V[s] = max_dict(Q[s])
    
print('policy:')
print_policy(policy, grid)

print("values:")
print_value(V, grid)

# what's the proportion of time we spend updating each part of Q?
print("update counts:")
total = np.sum(list(update_counts.values()))
for k in update_counts.keys():
    update_counts[k] =  float(update_counts[k]) / total
print_value(update_counts, grid)


rewards:
----------------------------
|-0.10 |-0.10 |-0.10 | 1.00 |
----------------------------
|-0.10 | 0.00 |-0.10 |-1.00 |
----------------------------
|-0.10 |-0.10 |-0.10 |-0.10 |
----------------------------
policy:
-----------------
| R | R | R |   |
-----------------
| U |   | U |   |
-----------------
| R | R | U | L |
-----------------
values:
----------------------------
| 0.61 | 0.80 | 1.00 | 0.00 |
----------------------------
| 0.40 | 0.00 | 0.80 | 0.00 |
----------------------------
| 0.31 | 0.46 | 0.62 | 0.45 |
----------------------------
update counts:
----------------------------
| 0.01 | 0.01 | 0.19 | 0.00 |
----------------------------
| 0.01 | 0.00 | 0.19 | 0.00 |
----------------------------
| 0.20 | 0.20 | 0.19 | 0.01 |
----------------------------
